<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/Alpha_RSI_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assets Tracking for Entries for Day Trading

In [1]:
pip install ccxt pandas numpy ta scikit-learn matplotlib seaborn requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 12.0 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ba377bf83d99e4f76d6113f4e0620ce380d8f7579c29318e3281ebcb6474d9b7
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [7]:
import ccxt
import pandas as pd
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator
from ta.volatility import AverageTrueRange
from ta.volume import OnBalanceVolumeIndicator

# Initialize exchange
exchange = ccxt.kucoin()  # Replace with your preferred exchange

# List of assets
symbols = [
    "BTC/USDT", "ETH/USDT", "AVAX/USDT", "SOL/USDT", "XAVA/USDT",
    "SLIM/USDT", "HNT/USDT", "NEAR/USDT", "DOGE/USDT", "SAND/USDT",
    "CRV/USDT", "CFX/USDT", "INJ/USDT", "LINK/USDT", "AAVE/USDT",
    "LTC/USDT", "ADA/USDT", "GAFI/USDT", "JUP/USDT", "MNT/USDT",
    "PYTH/USDT", "STG/USDT", "VET/USDT" ,
]

# Function to fetch historical data
def fetch_ccxt_data(symbol, timeframe, limit=500):
    """Fetch historical OHLCV data using CCXT."""
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=["time", "open", "high", "low", "close", "volume"])
    df["time"] = pd.to_datetime(df["time"], unit="ms")
    return df

# Function to calculate RSI, EMA, ATR, and OBV
def calculate_indicators(df, rsi_period=14, ema_period=200, atr_period=14):
    """Add RSI, EMA, ATR, and OBV indicators to the DataFrame."""
    df["RSI"] = RSIIndicator(df["close"], window=rsi_period).rsi()
    df["EMA"] = EMAIndicator(df["close"], window=ema_period).ema_indicator()
    df["ATR"] = AverageTrueRange(df["high"], df["low"], df["close"], window=atr_period).average_true_range()
    df["OBV"] = OnBalanceVolumeIndicator(df["close"], df["volume"]).on_balance_volume()
    return df

# Signal tracking function
def track_signals():
    """Track assets that meet the criteria."""
    signals = []

    for symbol in symbols:
        try:
            print(f"Processing {symbol}...")

            # Fetch data for required timeframes
            data_15m = fetch_ccxt_data(symbol, "15m")
            data_1h = fetch_ccxt_data(symbol, "1h")
            data_1d = fetch_ccxt_data(symbol, "1d")

            # Calculate indicators
            data_15m = calculate_indicators(data_15m)
            data_1h = calculate_indicators(data_1h)
            data_1d = calculate_indicators(data_1d)

            # Check RSI condition
            rsi_15m = data_15m["RSI"].iloc[-1] < 40
            rsi_1h = data_1h["RSI"].iloc[-1] < 40
            rsi_1d = data_1d["RSI"].iloc[-1] < 30

            # Signal criteria
            if (rsi_15m or rsi_1h or rsi_1d):
                signals.append({
                    "symbol": symbol,
                    "RSI_15m": data_15m["RSI"].iloc[-1],
                    "RSI_1h": data_1h["RSI"].iloc[-1],
                    "RSI_1d": data_1d["RSI"].iloc[-1],
                    "ATR_1d": data_1d["ATR"].iloc[-1],
                    "ATR_1h": data_1h["ATR"].iloc[-1],
                    "ATR_15m": data_15m["ATR"].iloc[-1],
                    "OBV_1d": data_1d["OBV"].iloc[-1],
                    "OBV_1h": data_1h["OBV"].iloc[-1],
                    "OBV_15m": data_15m["OBV"].iloc[-1],
                    "Price": data_1d["close"].iloc[-1],
                })

        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    return signals

# Main execution
if __name__ == "__main__":
    signals = track_signals()
    if signals:
        print("\nSignals Found:")
        for signal in signals:
            print(signal)
    else:
        print("\nNo signals found based on the given criteria.")


Processing BTC/USDT...
Processing ETH/USDT...
Processing AVAX/USDT...
Processing SOL/USDT...
Processing XAVA/USDT...
Processing SLIM/USDT...
Processing HNT/USDT...
Processing NEAR/USDT...
Processing DOGE/USDT...
Processing SAND/USDT...
Processing CRV/USDT...
Processing CFX/USDT...
Processing INJ/USDT...
Processing LINK/USDT...
Processing AAVE/USDT...
Processing LTC/USDT...
Processing ADA/USDT...
Processing GAFI/USDT...
Processing JUP/USDT...
Processing MNT/USDT...
Processing PYTH/USDT...
Processing STG/USDT...
Processing VET/USDT...

Signals Found:
{'symbol': 'BTC/USDT', 'RSI_15m': 38.775116269318524, 'RSI_1h': 44.847847122058546, 'RSI_1d': 59.26148516195349, 'ATR_1d': 4315.7539357089745, 'ATR_1h': 732.3643525373254, 'ATR_15m': 260.9697930773585, 'OBV_1d': 150749.7012496498, 'OBV_1h': -2663.4589565899455, 'OBV_15m': 909.0669284700008, 'Price': 104314.4}
{'symbol': 'ETH/USDT', 'RSI_15m': 39.35844588910437, 'RSI_1h': 41.27135887781358, 'RSI_1d': 47.085803228553935, 'ATR_1d': 187.37185244